In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')

In [ ]:
num_beams = 2
num_return_sequences = 2

paraphrase_questions = []

for i in train['question_en_origin']:
  paraphrase_questions.append(get_response(i, num_return_sequences, num_beams))

paraphrase_answers = []

for i in train['answer_en_origin']:
  paraphrase_answers.append(get_response(i, num_return_sequences, num_beams))

In [ ]:
df_question = pd.DataFrame(paraphrase_questions, columns = ['question_AUT_EN_1', 'question_AUT_EN_2'])
df_answer = pd.DataFrame(paraphrase_answers, columns = ['answer_AUT_EN_1', 'answer_AUT_EN_2'])

In [ ]:
frames = [df_question, df_answer]

result_1 = pd.concat(frames, axis = 1)

df = pd.concat([train.reset_index(drop=True), result_1.reset_index(drop=True)], axis=1)
df.head()

,Unnamed: 0.1,Unnamed: 0,id_qa,corpus,question_en_origin,question_pt_origin,question_en_paraphase,question_pt_paraphase,answer_en_origin,answer_pt_origin,...,question_meaningful,answer_equivalent,question_type,abstract_translated_pt,pt_question_translated_to_en,at_labels,question_AUT_EN_1,question_AUT_EN_2,answer_AUT_EN_1,answer_AUT_EN_2
0,0,0,A160,1,What method can be used to estimate air–sea se...,Qual método pode ser usado para estimar os flu...,To estimate sensitive air-sea sensible and lat...,Para estimar fluxos ar-mar de calor sensíveis ...,The turbulent covariance method.,O método de covariância turbulenta.,...,5.0,5.0,What,Os fluxos de calor do ar turbulento foram calc...,Which method can be used to estimate sensitive...,1.0,How can we estimate air–sea sensible- and late...,How can we estimate air–sea sensible– and late...,The method of covariance.,The method is turbulent.
1,1,1,B2252,2,Why is mercury considered one of the ten most ...,Porque o mercúrio é considerado uma das dez su...,What's the reason mercury is considered one of...,Por quais motivos o mercurio é considerado uma...,"Because in humans, exposure can be harmful to ...","Pois em humanos, a exposição pode ser prejudic...",...,4.0,4.0,Why,A saúde humana pode ser afetada por muitos asp...,Why is mercury considered one of the ten most ...,1.0,Mercury is one of the most toxic substances.,Why is mercury considered one of the most toxi...,Exposure to this neurotoxin can be harmful to ...,Exposure to this neurotoxin can be harmful to ...
2,2,2,B2044,2,What are the biological benefits of marine hyd...,Quais são os benefícios biológicos de hidratos...,NaN,NaN,Marine hydrates with seeping gas are high biod...,Hidratos marinhos com escape de gás são ambien...,...,3.0,4.0,What,Os hidratos marinhos (principalmente hidratos ...,What are the biological benefits of marine hyd...,NaN,What are the benefits of the marine environment?,What are the benefits of marine lysis?,Marine liquefaction is high biodiverse environ...,Marine liquefaction is high biodiverse.
3,3,3,A931,1,What is the most productive oil province in Br...,Qual é a província petrolífera mais produtiva ...,NaN,NaN,The Campos basin,A bacia de Campos,...,5.0,5.0,What,"Em 1939, o campo de óleo de Lobito, o primeiro...",What is the most productive oil province in Br...,1.0,What is the most productive oil province in Br...,What is the most productive oil region in Brazil?,The basin is called the Campos.,The basin of the Campos.
4,4,4,B2045,2,What is the importance of the cultural practic...,Qual a importância das práticas culturais rela...,What is the trade-off of cultural practices re...,Qual o retorno das práticas culturais relacion...,They are essential to the maintenance and crea...,Elas são essenciais para a manutenção e criaçã...,...,5.0,4.0,What,Serviços culturais e outros benefícios sociais...,How important is cultural practices related to...,1.0,The cultural practices related to the ocean ar...,The cultural practices related to the ocean ar...,They are essential to the creation of social c...,They are essential to the creation of social c...


In [ ]:
df.to_csv("parafrases_en_pegasus.csv")